# Create relationship Same_Visitor for Visitors this Year to Visitor BVA and LSE last year and Relationship attended_session

## Needed Files

- data/bva/output/scan_bva_past.csv
- data/bva/output/df_reg_demo_last_bva.csv
- data/bva/output/df_reg_demo_last_lva.csv
- data/bva/output/scan_lva_past.csv

In [ ]:
from neo4j import GraphDatabase
import pandas as pd


def get_node_attributes(uri, user, password, label, property_name, property_value):
    """
    Retrieves attributes of a node in Neo4j using the Python client.

    Args:
        uri: The Neo4j URI (e.g., "bolt://localhost:7687").
        user: The Neo4j username.
        password: The Neo4j password.
        label: The node label.
        property_name: The property name to filter by.
        property_value: The property value to filter by.

    Returns:
        A list of dictionaries, where each dictionary represents a node's attributes, or None if an error occurs.
    """
    try:
        with GraphDatabase.driver(uri, auth=(user, password)) as driver:
            with driver.session() as session:
                query = (
                    "MATCH (n:" + label + " {" + property_name + ": $property_value})"
                    "RETURN properties(n) AS properties"
                )
                result = session.run(query, property_value=property_value)
                return [record["properties"] for record in result]

    except Exception as e:
        print(f"An error occurred: {e}")
        return None


def get_specific_attributes(
    uri, user, password, label, property_name, property_value, return_properties
):

    try:
        with GraphDatabase.driver(uri, auth=(user, password)) as driver:
            with driver.session() as session:
                return_string = ", ".join(["n." + prop for prop in return_properties])
                query = (
                    "MATCH (n:" + label + " {" + property_name + ": $property_value})"
                    "RETURN " + return_string
                )
                result = session.run(query, property_value=property_value)
                return [dict(record) for record in result]

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [ ]:
# <elementId>: 4:d57c3dfd-a63c-4476-9412-d40330b735c5:58
# <id>: 58
# BadgeId: EDSPSTY
# BadgeId_last_year: 2LQCU8F
# BadgeType: Delegate
# Country: UK
# Days_since_registration: 119
# Email: jo.gale@effem.com
# Email_domain: effem.com
# ShowRef: BVA2025
# Source: BVA Key Stakeholders
# assist_year_before: 1
# job_role: NA
# organisation_type: NA
# what_type_does_your_practice_specialise_in: NA

# <elementId>: 5:d57c3dfd-a63c-4476-9412-d40330b735c5:1152922604118474810
# <id>: 1152922604118474810

In [ ]:
uri = "bolt://127.0.0.1:7687"
user = "neo4j"
password = ""  # Replace with your password.
label = "Sessions_this_year"  # BadgeId: QNZ9L2P
property_name = "session_id"
property_value = "49679"

all_attributes = get_node_attributes(
    uri, user, password, label, property_name, property_value
)

if all_attributes:
    print("All attributes:")
    for attributes in all_attributes:
        print(attributes)

In [ ]:
with GraphDatabase.driver(uri, auth=(user, password)) as driver:
    with driver.session() as session:
        query = (
            "MATCH (n:" + label + " {" + property_name + ": $property_value})"
            "RETURN properties(n) AS properties"
        )
        result = session.run(query, property_value=property_value)
        print([record["properties"] for record in result])

In [ ]:
specific_attributes = get_specific_attributes(
    uri, user, password, label, property_name, property_value, ["title"]
)

if specific_attributes:
    print("\nSpecific attributes:")
    for attributes in specific_attributes:
        print(attributes)

# Connect BVA this year with BVA past year

```
# Visitor_this_year
<elementId>: 4:d57c3dfd-a63c-4476-9412-d40330b735c5:15
<id>: 15
BadgeId: HSIPH6L
BadgeId_last_year_bva: W2UTV25
BadgeId_last_year_lva: NA
BadgeType: Delegate
Company: The Vet Connection
Country: UK
Days_since_registration: 79
Email: kirsty.cavill@thevetconnection.co.uk
Email_domain: thevetconnection.co.uk
JobTitle: Director of Veterinary Services
ShowRef: BVA2025
Source: EmailLimitedBursaryPushMarch
assist_year_before: 1
job_role: Clinical or other Director
organisation_type: Other
what_type_does_your_practice_specialise_in: Small Animal

# Visitor_last_year_bva

<elementId>: 4:d57c3dfd-a63c-4476-9412-d40330b735c5:1461
<id>: 1461
BadgeId: BLBZF9H
BadgeType: Delegate
Company: Foxhall Veterinary Clinic
Country: UK
Days_since_registration: 8
Email: danieljohnmullen@gmail.com
Email_domain: gmail.com
JobTitle: Veterinary Student Nurse
ShowRef: BVA2024
Source: Reid
job_role: NA
organisation_type: NA
what_areas_do_you_specialise_in: NA
```

In [ ]:
from neo4j import GraphDatabase


def create_same_visitor_relationships_bva(uri, user, password):
    """
    Creates Same_Visitor relationships for all Visitor_this_year nodes where BadgeId_last_year != "NA".
    """
    driver = GraphDatabase.driver(uri, auth=(user, password))

    with driver.session() as session:
        query = """
            MATCH (this_year:Visitor_this_year)
            WHERE this_year.BadgeId_last_year_bva <> "NA"
            MATCH (last_year:Visitor_last_year_bva {BadgeId: this_year.BadgeId_last_year_bva})
            CREATE (this_year)-[:Same_Visitor {type : 'bva'}]->(last_year)
        """
        try:
            result = session.run(query)
            print("Same_Visitor relationships created successfully!")
        except Exception as e:
            print(f"Error creating relationships: {e}")

    driver.close()

In [ ]:
create_same_visitor_relationships_bva(uri, user, password)

# Connect BVA this year with LVA  past year

```
# Visitor_this_year
<elementId>: 4:d57c3dfd-a63c-4476-9412-d40330b735c5:15
<id>: 15
BadgeId: HSIPH6L
BadgeId_last_year_bva: W2UTV25
BadgeId_last_year_lva: NA
BadgeType: Delegate
Company: The Vet Connection
Country: UK
Days_since_registration: 79
Email: kirsty.cavill@thevetconnection.co.uk
Email_domain: thevetconnection.co.uk
JobTitle: Director of Veterinary Services
ShowRef: BVA2025
Source: EmailLimitedBursaryPushMarch
assist_year_before: 1
job_role: Clinical or other Director
organisation_type: Other
what_type_does_your_practice_specialise_in: Small Animal

# Visitor_last_year_lva

<elementId>: 4:d57c3dfd-a63c-4476-9412-d40330b735c5:1869
<id>: 1869
BadgeId: MXHVTQE
BadgeType: Delegate
Company: British Veterinary Association
Country: UK
Days_since_registration: -148
Email: nicky.paull@btinternet.com
Email_domain: btinternet.com
JobTitle: BVA Board member and Past President
ShowRef: LVS2024
Source: BVA
job_role: Clinical or other Director
organisation_type: Other
what_areas_do_you_specialise_in: Companion Animal
```

In [ ]:
from neo4j import GraphDatabase


def create_same_visitor_relationships_lva(uri, user, password):
    """
    Creates Same_Visitor relationships for all Visitor_this_year nodes where BadgeId_last_year != "NA".
    """
    driver = GraphDatabase.driver(uri, auth=(user, password))

    with driver.session() as session:
        query = """
            MATCH (this_year:Visitor_this_year)
            WHERE this_year.BadgeId_last_year_lva <> "NA"
            MATCH (last_year:Visitor_last_year_lva {BadgeId: this_year.BadgeId_last_year_lva})
            CREATE (this_year)-[:Same_Visitor {type : 'lva'}]->(last_year)
        """
        try:
            result = session.run(query)
            print("Same_Visitor relationships created successfully!")
        except Exception as e:
            print(f"Error creating relationships: {e}")

    driver.close()

In [ ]:
create_same_visitor_relationships_lva(uri, user, password)

# Scan Data Visitors BVA and LVA Past Year. Use the Demo data with only badgeID of user which attended BVA this and BVA past or BVA this and LVA past

In [ ]:
scan_bva_past = pd.read_csv("data/bva/output/scan_bva_past.csv")
scan_bva_past.columns

In [ ]:
df_reg_demo_last_bva = pd.read_csv("data/bva/output/df_reg_demo_last_bva.csv")
df_reg_demo_last_lva = pd.read_csv("data/bva/output/df_reg_demo_last_lva.csv")

In [ ]:
list_reg_demo_last_bva = list(df_reg_demo_last_bva["BadgeId"].unique())
list_reg_demo_last_lva = list(df_reg_demo_last_lva["BadgeId"].unique())
len(list_reg_demo_last_bva), len(list_reg_demo_last_lva)

In [ ]:
scan_bva_past = scan_bva_past[scan_bva_past["Badge Id"].isin(list_reg_demo_last_bva)]
len(scan_bva_past)

In [ ]:
import pandas as pd
from neo4j import GraphDatabase


def create_attended_session_relationships_neo4j_bva(df, uri, user, password):
    """
    Creates 'attended_session' relationships in Neo4j based on DataFrame data.

    Args:
        df (pd.DataFrame): DataFrame containing visitor scan data.
        uri (str): Neo4j connection URI.
        user (str): Neo4j username.
        password (str): Neo4j password.
    """

    driver = GraphDatabase.driver(uri, auth=(user, password))

    def create_relationships(tx, row):
        query = """
        MATCH (v:Visitor_last_year_bva {BadgeId: $visitor_badge_id})
        MATCH (s:Sessions_past_year {key_text: $session_key_text})
        CREATE (v)-[:attended_session {scan_date: $scan_date, file: $file, seminar_name: $seminar_name}]->(s)
        """
        tx.run(
            query,
            visitor_badge_id=row["Badge Id"],
            session_key_text=row["key_text"],
            scan_date=str(row["Scan Date"]),
            file=row["File"],
            seminar_name=row["Seminar Name"],
        )

    with driver.session() as session:
        for _, row in df.iterrows():
            session.execute_write(create_relationships, row)

    driver.close()

In [ ]:
create_attended_session_relationships_neo4j_bva(scan_bva_past, uri, user, password)

In [ ]:
scan_lva_past = pd.read_csv("data/bva/output/scan_lva_past.csv")
scan_lva_past.columns

In [ ]:
scan_lva_past = scan_lva_past[scan_lva_past["Badge Id"].isin(list_reg_demo_last_lva)]
len(scan_lva_past)

In [ ]:
def create_attended_session_relationships_neo4j_lva(df, uri, user, password):
    """
    Creates 'attended_session' relationships in Neo4j based on DataFrame data.

    Args:
        df (pd.DataFrame): DataFrame containing visitor scan data.
        uri (str): Neo4j connection URI.
        user (str): Neo4j username.
        password (str): Neo4j password.
    """

    driver = GraphDatabase.driver(uri, auth=(user, password))

    def create_relationships(tx, row):
        query = """
        MATCH (v:Visitor_last_year_lva {BadgeId: $visitor_badge_id})
        MATCH (s:Sessions_past_year {key_text: $session_key_text})
        CREATE (v)-[:attended_session {scan_date: $scan_date, file: $file, seminar_name: $seminar_name}]->(s)
        """
        tx.run(
            query,
            visitor_badge_id=row["Badge Id"],
            session_key_text=row["key_text"],
            scan_date=str(row["Scan Date"]),
            file=row["File"],
            seminar_name=row["Seminar Name"],
        )

    with driver.session() as session:
        for _, row in df.iterrows():
            session.execute_write(create_relationships, row)

    driver.close()

In [ ]:
create_attended_session_relationships_neo4j_lva(scan_lva_past, uri, user, password)